In [8]:
from pymilvus import MilvusClient, model

default_embedding_model = model.DefaultEmbeddingFunction()
query = default_embedding_model.encode_queries(["物业擅自提高物业费，是否违反民法典中的法律？"])[0]
default_embedding_dimension = len(query)

In [9]:
default_embedding_dimension

768

In [10]:
milvus_client = MilvusClient(uri="./milvus_demo.db")
collection_name = "mfd_collection"
if milvus_client.has_collection(collection_name):
    milvus_client.drop_collection(collection_name)
milvus_client.create_collection(
    collection_name=collection_name,
    dimension=default_embedding_dimension,
    metric_type="IP",  # 内积距离
    consistency_level="Strong"  # 强一致性
)

In [11]:
with open("mfd.md", "r", encoding="utf-8") as f:
    data = list(f.readlines())
len(data)

1113

In [12]:
docs = [ { "id": i, "text": text, "vector": vector } for i, (text, vector) in enumerate(zip(data, default_embedding_model.encode_documents(data))) ]
milvus_client.insert(collection_name, docs)

{'insert_count': 1113, 'ids': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215,

In [20]:
question = "甲方违反合同怎么办"
query_data = default_embedding_model.encode_queries([question])
query_data

[array([-9.93222058e-03,  4.68229764e-02, -3.67522755e-02, -8.06826553e-02,
        -2.61728407e-02,  3.73490318e-03,  9.90155462e-03, -8.76287526e-02,
        -1.55368180e-02,  5.37704861e-02, -8.65141838e-02, -1.18772502e-02,
         4.30214888e-02, -2.44343250e-02,  1.66521767e-02,  2.23483081e-02,
        -3.18444377e-02, -4.02236764e-03,  2.29590701e-02, -4.86642133e-02,
        -3.80949404e-03, -1.59919867e-03,  2.32126944e-02, -3.35544044e-02,
        -3.71057093e-02,  1.86744156e-03,  1.36478595e-02, -2.56726893e-02,
        -2.49355981e-02,  3.94119431e-02,  1.63484775e-02,  4.86203474e-02,
         8.02349119e-02, -4.49607014e-02, -7.76415416e-02, -5.16100847e-02,
        -3.21352926e-02,  3.18942416e-02,  1.29663163e-02, -5.76224895e-02,
         8.74985437e-02,  4.73525719e-02,  2.03682095e-02,  4.44319557e-02,
        -3.21467054e-02, -1.22552114e-02, -1.86714735e-02, -1.22510570e-02,
        -5.41150572e-03, -3.72957509e-03, -1.92851604e-02, -6.46768241e-02,
        -2.8

In [21]:
search_result = milvus_client.search(
    collection_name=collection_name,
    data=query_data,
    limit=3,
    output_fields=["text"],
    consistency_level="Strong"
)
search_result

data: [[{'id': 285, 'distance': 1.0, 'entity': {'text': '共同共有人对共有财产享有平等的占有、使用、收益和处分的权利。\n'}}, {'id': 218, 'distance': 1.0, 'entity': {'text': '管理规约对全体业主具有约束力。\n'}}, {'id': 42, 'distance': 1.0, 'entity': {'text': '不动产登记簿应当采用纸质形式或者电子形式。\n'}}]]

In [22]:
search_result[0]

[{'id': 285, 'distance': 1.0, 'entity': {'text': '共同共有人对共有财产享有平等的占有、使用、收益和处分的权利。\n'}}, {'id': 218, 'distance': 1.0, 'entity': {'text': '管理规约对全体业主具有约束力。\n'}}, {'id': 42, 'distance': 1.0, 'entity': {'text': '不动产登记簿应当采用纸质形式或者电子形式。\n'}}]

In [23]:
context = "\n".join(map(lambda x: x['entity']['text'], search_result[0]))
context

'共同共有人对共有财产享有平等的占有、使用、收益和处分的权利。\n\n管理规约对全体业主具有约束力。\n\n不动产登记簿应当采用纸质形式或者电子形式。\n'

In [24]:
SYSTEM_PROMPT = """
Human: 你是一个专业的律师。你能用你的知识和我提供的民法典法条为我提供专业的法律咨询。
"""
USER_PROMPT = f"""
请使用以下用 <context> 标签括起来的民法典法条来回答用 <question> 标签括起来的问题。
<context>
{context}
</context>
<question>
{question}
</question>
"""
USER_PROMPT

'\n请使用以下用 <context> 标签括起来的民法典法条来回答用 <question> 标签括起来的问题。\n<context>\n共同共有人对共有财产享有平等的占有、使用、收益和处分的权利。\n\n管理规约对全体业主具有约束力。\n\n不动产登记簿应当采用纸质形式或者电子形式。\n\n</context>\n<question>\n甲方违反合同怎么办\n</question>\n'

In [25]:
from openai import OpenAI
import os
deepseek_client = OpenAI(
    api_key=os.getenv("DEEPSEEK_API_KEY"),
    base_url="https://api.deepseek.com/v1",
)

In [26]:
response = deepseek_client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": USER_PROMPT},
    ],
)
response

ChatCompletion(id='e4a6059e-3dbe-406a-821d-e94a3433c02e', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='根据您提供的民法典法条内容，您的问题涉及的是合同违约问题，但您提供的法条主要涉及共有财产、管理规约和不动产登记等事项，与合同违约没有直接关联。\n\n针对"甲方违反合同怎么办"的问题，虽然您提供的法条不适用，但根据民法典相关规定，我可以给出以下专业建议：\n\n1. 首先应当审查合同条款，看是否有关于违约责任的明确约定。\n\n2. 根据民法典合同编相关规定，您可以：\n- 要求甲方继续履行合同\n- 采取补救措施\n- 要求赔偿损失\n\n3. 具体救济方式取决于：\n- 违约的严重程度\n- 合同是否还能继续履行\n- 造成的实际损失情况\n\n4. 建议步骤：\n(1) 先与甲方协商解决\n(2) 协商不成可申请调解\n(3) 必要时可向法院提起诉讼\n\n如需更具体的法律意见，建议您提供：\n- 合同的具体违约条款\n- 甲方违约的具体事实\n- 造成的实际损失情况\n\n这样我可以根据民法典合同编的相关规定给出更有针对性的法律建议。', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None))], created=1749305802, model='deepseek-chat', object='chat.completion', service_tier=None, system_fingerprint='fp_8802369eaa_prod0425fp8', usage=CompletionUsage(completion_tokens=230, prompt_tokens=110, total_tokens=340, completion_tokens_details=None, prompt_tokens_details=PromptTokensDetails(audio_tokens=None

In [27]:
response.choices[0].message.content

'根据您提供的民法典法条内容，您的问题涉及的是合同违约问题，但您提供的法条主要涉及共有财产、管理规约和不动产登记等事项，与合同违约没有直接关联。\n\n针对"甲方违反合同怎么办"的问题，虽然您提供的法条不适用，但根据民法典相关规定，我可以给出以下专业建议：\n\n1. 首先应当审查合同条款，看是否有关于违约责任的明确约定。\n\n2. 根据民法典合同编相关规定，您可以：\n- 要求甲方继续履行合同\n- 采取补救措施\n- 要求赔偿损失\n\n3. 具体救济方式取决于：\n- 违约的严重程度\n- 合同是否还能继续履行\n- 造成的实际损失情况\n\n4. 建议步骤：\n(1) 先与甲方协商解决\n(2) 协商不成可申请调解\n(3) 必要时可向法院提起诉讼\n\n如需更具体的法律意见，建议您提供：\n- 合同的具体违约条款\n- 甲方违约的具体事实\n- 造成的实际损失情况\n\n这样我可以根据民法典合同编的相关规定给出更有针对性的法律建议。'